In [1]:
import json
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

I1116 21:15:31.726818 15756 file_utils.py:32] TensorFlow version 2.0.0 available.
I1116 21:15:32.399328 15756 file_utils.py:39] PyTorch version 1.3.1+cpu available.
W1116 21:15:32.411558 15756 __init__.py:28] To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html
I1116 21:15:32.545044 15756 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
with open('data/test.json', 'r', encoding='utf8') as f:
    data = json.load(f)

In [3]:
question_list = []
paragraphs_list = []

In [4]:
for obj in data:
    question_list.append(obj['question'])
    paragraphs_list.append([paragraph['text'] for paragraph in obj['paragraphs']])

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertForSequenceClassification.from_pretrained('./model/')

I1116 21:15:35.473552 15756 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at C:\Users\DELL\.cache\torch\transformers\96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
I1116 21:15:35.775254 15756 configuration_utils.py:148] loading configuration file ./model/config.json
I1116 21:15:35.777766 15756 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "poole

In [6]:
i = 0

In [7]:
print(f'Question {i + 1}/{len(question_list)}: {question_list[i]} \n')
for (idx, paragraph) in enumerate(paragraphs_list[i]):
    inputs = tokenizer.encode_plus(question_list[i], paragraph, add_special_tokens=True, return_tensors='tf')
    logits = model(inputs['input_ids'], token_type_ids=inputs['token_type_ids'])[0]
    softmax = tf.math.softmax(logits).numpy()[0]
    answer = bool(logits.numpy().argmax())
    print(f'Paragraph {idx + 1}: {paragraph} - Answer: {answer} - Relevance score: {softmax[1]} \n')
    
i += 1
i %= len(question_list)

Question 1/501: Toà nhà nào cao nhất Việt Nam 

Paragraph 1: The Landmark 81 là một toà nhà chọc trời trong tổ hợp dự án Vinhomes Tân Cảng , một dự án có tổng mức đầu tư 40.000 tỷ đồng , do Công ty Cổ phần Đầu tư xây dựng Tân Liên Phát thuộc Vingroup làm chủ đầu tư . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . - Answer: True - Relevance score: 0.6124688386917114 

Paragraph 2: Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn . Dự án được khởi công ngày 26/07/2014 . - Answer: True - Relevance score: 0.9692904353141785 

